# Bayesian Network from Data
[//]: # (Summary)
In this tutorial we are creating a Bayesian Network (BN) for the find task of a Kill Chain as shown in the figure below.

![alt text](images/Ship-Wake-BN-Find.png)

We will walk through the aspectes of a BN and then demonstrate creating one using a CSV file to learn the conditional probability tables which define the edges of the graph. Finally, we demonstrate how the BN task is incorporated in mimik's `Killweb` module.

[//]: # (Objective)
By the end of this tutorial you will understand how a BN is created using a defined graph and associated CSV data file, how to infer with that BN, and how to run the `Killweb` module with this Find BN task.

This tutorial requires the Python package [pgmpy](https://pgmpy.org). Please ensure that pgmpy is installed by running the following command in a terminal with the mimik virtual environment active:
```
$ pip install pgmpy
```
Or, follow the [pgmpy installation instructions here.](https://pgmpy.org/started/base.html)

In [ ]:
import json
import warnings
import pandas as pd
import seaborn as sns
from tasks.component_BN import ComponentBN
from pgmpy.factors.discrete import TabularCPD
from mimik.killweb import Killweb

warnings.filterwarnings('ignore')

## Creating the Network

### Edges

Edges define the nodes and their relationship. For example, if there is a connection from node $\alpha$ to $\beta$ we specify that with a the tuple ( $\alpha$ , $\beta$ ).

In this example, the success of a radar and sea sensing platform to perform the Find task, by detecting possible targets via their ship wake on the ocian surface, is affected by its radar band and the sea height and direction, so our edges are:
- ('band', 'pred'),
- ('sea_height', 'pred'),
- ('sea_direction', 'pred'),

Edges are set in an `edges` variable that is a list of tuples for all the edges in the network.

In [ ]:
edges = [
    ('band', 'pred'),
    ('sea_height', 'pred'),
    ('sea_direction', 'pred')
]

### Data

The data for this Bayesian network (BN) is stored in a csv file. The nodes listed above correspond to column names in the csv and must be spelled the same way. The cell below loads the data as a pandas dataframe. We can see that there are a lot more columns than the four we use for the nodes. This is fine, as long as we choose the most important features for the outcome and properly model their relationship.

In [ ]:
data = pd.read_csv("data/processed.csv")
data

The data for this example is based on using a deep learning model to identify persistent ship wakes in sythetic aperture radar (SAR) images of the ocean surface. The data is based on [this publication](https://doi.org/10.2514/6.2024-2056). The band refers to one of three bands (or freqeuncy range) for SAR satellites. The presence or absence of a persistent wake is also influence by the height of waves (sea height) and the orientation of ocean currents (sea direction).

### Priors

In certain situations, especially in cases where the dataset is small, we want to specify a prior distribution for the Bayesian network. We won't go into the details on what exactly a prior is or the role it plays in Bayesian statistics. Instead we should think of this as a way to hedge the probabilities the BN will learn from the data and include some additional uncertainty. 

In cases where only one or two data points exist for any particular combination of input nodes we can end up with an BN that is over- or under-confident of the results. For example, if we have test data with only one result for each combination of band, sea_height, and sea_direction the BN will learn conditional probabilties where the pred value is either 1 or 0. However, we don't want to make such a strong inference -- can we really say the outcome will always be 100% true from a single test? The following cell stores parameters for a simple uniform prior. Additional information on priors can be learned [from the pgmpy documentation](https://pgmpy.org/detailed_notebooks/10.%20Learning%20Bayesian%20Networks%20from%20Data.html#Bayesian-Parameter-Estimation), which is how we implement the BN. 

If you wish to simply ignore using a prior, set `estimator="MaximumLikelihood"` and omit `prior_type` and `equivalent_sample_size` from the configs.

In [ ]:
estimator = "Bayesian"
prior_type = 'BDeu'
equivalent_sample_size = 10

### Constructing the BN

Finally, we create the BN using the `ComponentBN` class, which takes the edges, data, and prior distribution arguments as input to construct the BN.

In [ ]:
find_BN = ComponentBN(
    edges=edges,
    data=data,
    estimator=estimator,
    prior_type=prior_type,
    equivalent_sample_size=equivalent_sample_size
)

We can visualize the network by calling the `draw_network` method.

In [ ]:
find_BN.draw_network()

## Using the BN

### Inference Probability of Success

Given some evidence, `band="C"`, `sea_height=0.5`, and `sea_direction="Head"`, we can get the associated probability table for the Find node by calling the `get_infer` method:

In [ ]:
query = find_BN.get_infer(
    evidence={
        "band": "C",
        "sea_height": 0.5,
        "sea_direction": "Head"
    },
    variables=["pred"]
)
print(query)

We can also get just the probability of success using the `get_prob_success` method and providing the evidence and desired outcome `pred=1`: 

In [ ]:
find_BN.get_prob_success(
    evidence={
        "band": "C",
        "sea_height": 0.5,
        "sea_direction": "Head"
    },
    pred=1
)

### Sampling

While the above inference provides the probability of success, 1, (or failure, 0), the Monte Carlo simulation in MIMIK requires random sampling from the probability of outcomes for the Find node. This is done through the `get_sample` method, which does return a pandas data frame, but we can just look at the outcome. Given the high probability of success and the low sample size, the outcome is mostly likely 1.

In [ ]:
%%capture --no-stderr
df = find_BN.get_sample(
    evidence={
        "band": "C",
        "sea_height": 0.5,
        "sea_direction": "Head"
    },
    size=1
)
df

We can just grab the value of `pred` for the sample generated above.

In [ ]:
df["pred"].values[0]

If we increase the sample `size` we can get enough results to see failures and can plot a histogram of the results to visualize the ratio of success to failure.

In [ ]:
%%capture --no-stderr
results = find_BN.get_sample(
    evidence={
        "band": "C",
        "sea_height": 0.5,
        "sea_direction": "Head"
    },
    size=100_000
)["pred"]

In [ ]:
print(results.value_counts())

sns.histplot(results.values);

If we change the evidence for band to "S" and sea direction to "Calm", we can see that the probability of success drops significantly. By constructing and using BNs in mimik we can account for changes in conditions like this when modeling Killwebs.

In [ ]:
%%capture --no-stderr
results = find_BN.get_sample(
    evidence={
        "band": "S",
        "sea_height": 0.5,
        "sea_direction": "Calm"
    },
    size=100_000
)["pred"]

In [ ]:
print(results.value_counts())

sns.histplot(results.values);

## JSON Configs

This is the json configuration for this Bayesian network. Because it's constant for any similar Find task node in the Killweb we save this configuration in its own file in the `configs/` directory. This type of configuration is how users can create their own custom BN for new Killwebs. Each type of BN node will have its own json config file, separate from the Killweb config file. More details for the configuration files are discussed in the enxt section. This section is just to demonstrate what the BN configs looks like and how it compares to the construction we did in the previous section.

In [ ]:
text = """
{
    "edges": [
        ["band", "pred"],
        ["sea_height", "pred"],
        ["sea_direction", "pred"]
    ],
    "data": "data/processed.csv",
    "CPDs": null,
    "estimator": "Bayesian",
    "prior_type": "BDeu",
    "equivalent_sample_size": 10
}
"""

We can load the text above and provide it to the `ComponentBN` class to create the Find task node. Then we rerun the `draw_network` and `get_infer` methods to demonstrate the BN is the same as the one we created in the previous section.

In [ ]:
data = json.loads(text)

find_BN = ComponentBN(**data)

In [ ]:
find_BN.draw_network()

In [ ]:
q = find_BN.get_infer(
    evidence={
        "band": "C",
        "sea_height": 0.5,
        "sea_direction": "Head"
    },
    variables=["pred"]
)
print(q)

## Using the Killweb Module

All of the above setup and sampling of outcomes is automoated by the `Killweb` module in mimik. All the user needs to provide are the json configs for the Killweb and the associated Bayesian network tasks. For this tutorial, the correct configurations are already set up in the `configs` subdirectory. All we have to do is specify this working directory (`.`) and the main config file `configs/bn_killchain.json` ("killchain" becuase this tutorial only has a single path). The `task_arguments` for this Find task node must include the absolute path to the BN config file. We must also specify the name of the `outcome` node and the success `condition` so the `BN.get_sample` method can be called correctly during the Monte Carlo simulation. The remaining arguments under `task_arguments` are the evidence that are used for the inference and sampling.

```json
"task_arguments": {
    "BN_config": "/absolute/path/to/mimik/ship_wake_find_task/configs/find_simple.json",
    "outcome": "pred",
    "condition": 1,
    "band": "C",
    "sea_height": 0.5,
    "sea_direction": "Head"
}
```

The killweb module is called by specifying the working directory of this example case and the path from there to the killchain config file.

In [ ]:
killweb = Killweb(
    working_dir=".",
    config_file="configs/bn_killchain.json",
)

The data is read and confirmed that the configs are in valid json format. Next we can print all the paths through the killweb (there will be only one path in this tutorial -- "Radar_1, Sensor_1, Track Algorithm_1, Equation_1, Missle_1, Personnel_1"), and then run the Monte Carlo simulation with 1,000 iterations.

In [ ]:
killweb.print_all_paths_in_killweb()

In [ ]:
%%capture --no-stderr
killweb.monte_carlo_on_paths(1000)

Now that the Monte Carlo is complete, we can print the probabilities of success for the path in the killchain:

In [ ]:
killweb.print_probabilities_of_paths(1)

In cases where there are more than one path in the Killweb, we can specify the path to test and get its probability of success as the proportion of time it sucessfully complete:

In [ ]:
path_to_test = ["Radar_1", "Sensor_1", "Track Algorithm_1", "Equation_1", "Missle_1", "Personnel_1"]
killweb.print_proportion_complete(path_to_test)

For a specific path, we can also plot the distribution of outcomes for each node as both a proportion of binary success using the `killweb.plot_monte_carlo_distribution` method or as the histogram of actual probabilities of success using the `killweb.plot_probability_distribution` method. Not that for the `killweb.plot_probability_distribution`, becuase all the probabilities are high it will look like outcomes are either 1.0 or 0.